# Expectation-Maximization for the Gaussian Mixture Model

---

In this tutiorial we will develop the Expectation-Maximization (EM) algorithm, and some of its statistic variants, in the context of Gaussian Mixture Model (GMM).

---

_**NB:** If you need to install a package in `Python` you may use the command `!pip install <package-name>` in a **code** cell._


<small>**Credits:** This tutorial is based on [zfeng](https://medium.com/@zhe.feng0018/coding-gaussian-mixture-model-and-em-algorithm-from-scratch-f3ef384a16ad)'s' one.</small>

## Preliminaries

A GMM is useful for modeling data that comes from one of several groups: the groups might be different from each other, but data points within the same group can be well-modeled by a Gaussian distribution. The main issue is to estimate the parameters of the mixture, _i.e_ to find the most likely ones. Moreover, we aim to determine if our sample follow a Gaussian mixture distribution or not.

Let consider a $n$-sample $Y=(Y_1,\,\ldots,\,Y_n)$ of $\mathbb{R}^d$. For each individual, we observe a random variable $Y_i$ and assume there is an unobserved variable $Z_i$ for each person which encode the class of $Y_i$. More formally, we consider a mixture of $K$ Gaussian: For all $i\in\{1,\ldots,n\}$ and $k\in\{1,\ldots,K\}$,
$$ 
\left\{\begin{aligned}
	& Z_i \,;\, \theta ~\sim~ \sum_{k=1}^K \alpha_k\, \delta_k \,, \\
	& Y_i ~\vert~ \{Z_i=k\} \,;\, \theta ~\sim~ \mathcal{N}(\mu_k,\Sigma_k) \,,
\end{aligned}\right.
$$
where
* $\alpha=(\alpha_1,\ldots,\alpha_K)\in\mathbb{R}_+^K$ checks $\sum_{k=1}^K\alpha_k=1$ and denotes the _mixing distribution_,
* $\mu=(\mu_1,\ldots,\mu_K)\in\mathbb{R}^{dK}$ denotes the set of _means_, and
* $\Sigma=(\Sigma_1,\ldots,\Sigma_K)\in(\mathcal{S}_d^+)^{K}$ denotes the set of _covariance matrices_.

Unless otherwise stated, we suppose that $K$ is fixed.

In [16]:
import numpy as np
import pandas as pd
import random as rd

import matplotlib.pyplot as plt
import numpy.linalg as la
#import scipy.special as sp

## Gaussian Mixture Model

##### <span style="color:purple">**Question:** What are the paramters of the model ?</span>

> Answer

##### <span style="color:purple">**Todo:** Write down the likelihood of the model.</span>

Based on the two `gaussian_pdf` and `component_pdfs` functions provided, write a `likelihood_function` that calculates the likelihood for this mixture model.

In [2]:
def gaussian_pdf(x, mu, sigma):
    # x vector should be in column, but numpy treat 1-d array as vector so can save the extra step
    n = len(x)
    return (2*np.pi)**(-n/2) * (la.det(sigma) ** (-1/2) ) * np.exp(-1/2*((x-mu).T@la.inv(sigma)@(x-mu)))

In [3]:
def component_pdfs(x, mus, sigmas):
    """
    The component pdf p_k(x; mu_k, sigma_k)
        :param mus: Kxn array (n be the dimension of x vector and K be the num of components), mean of k n-dim gaussian distr.
        :param sigmas: Knxn array covariance of k n-dim gaussian distr.
        :return: K-dim array contain probability of each component pdf
    """
    n_components = mus.shape[0]
    return np.array([gaussian_pdf(x, mus[k,:], sigmas[k, :, :]) for k in range(n_components)])

In [4]:
## TO BE COMPLETED ##

def likelihood_function(X, taus, mus, sigmas):
    """
    The component pdf p_k(x; mu_k, sigma_k)
        :param taus: K-dim array contains the weight (or prior of hidden var) of each gaussian component
        :param mus: Kxn array (n be the dimension of x vector and K be the num of components), mean of k n-dim gaussian distr.
        :param sigmas: Knxn array covariance of k n-dim gaussian distr.
        :return: numeric between 0 and 1, the likelihood function value
    """

    [...]

In [5]:
# %load solutions/likelihood_function.py

##### <span style="color:purple">✍️ **Todo:** Write down the complete log-likelihood of the model.</span>

Prove that le complete log-likelihood of the  model writes: For all $y\in\mathbb{R}^d$ and $z\in\{1,\ldots,K\}$, 
$$
\log q(y,\,z\,;\,\theta) 
    \,=\, \sum_{i=1}^n \sum_{k=1}^K \log\left( f_{\mathcal{N}(\mu_k,\sigma_k)}(y_i) \right) \mathbb{1}_{\{z_i=k\}}
    \,+\, \sum_{i=1}^n \sum_{k=1}^K \log(\alpha_k) \mathbb{1}_{\{z_i=k\}} \,.
$$

> **Exercise**

## E-step

##### <span style="color:purple">✍️ **Todo:** Compute the conditional expectation</span>
$$
Q(\theta \,\vert\, \theta^{(t)}) \,=\, \mathbb{E}_{Z\,\sim\,p(\cdot\vert y\,;\,\theta^{(t)})} \left[\, \log q(y,\,z\,;\,\theta) \,\vert\, y \,;\, \theta^{(t)} \,\right]
$$

1.  Justify that the $E$-step amounts to compute $\tau_{i,k} = \tau_{i,k}^{(t)} := \mathbb{P}\left(Z_i=k \,\vert\, y_i\,;\,\theta^{(t)}\right)$ for all $i$ and $k$.
2.  Using the Bayes rule, writes the $(\tau_{i,k})_{i\in\{1,\ldots,n\},\, i\in\{1,\ldots,K\}}$ in a closed form.

> **Exercise**

##### <span style="color:purple">**Todo:** Write an 'e_step' function to perform the E-step.</span>

In [6]:
## TO BE COMPLETED ##

def e_step(X, taus, mus, sigmas):
    """
        E step of the EM algorithm, caculates the posterior T_{k, i}=P(z_i=k|y_i)
        it returns T_{k,i} in the form of a KxN T matrix where each element is T_{k, i}
        :param X: Nxn matrix represents N number of n-dim data points
        :param taus: K-dim vector, the weight of each component, or the prior of the hidden stats z
        :param mus: Kxn matrix (n be the dimension of x vector and K be the num of components), mean of k n-dim gaussian distr.
        :param sigmas: Kxnxn matrix covariance of k n-dim gaussian distr.
        :return: T_{k,i} in the form of a KxN T matrix where each element is T_{k, i}
    """

    [...]

In [7]:
# %load solutions/e_step.py

## M-step

##### <span style="color:purple">✍️ **Todo:** Maximize $Q(\,\cdot\,\vert\, \theta^{(t)})$ under the constraint that $\sum_{k=1}^K\alpha_k=1$</span>

1. For all $k\in\{1,\ldots,K\}$, find the maximum in $\alpha_k$ of $Q(\,\cdot\,\vert\, \theta^{(t)})$.
		
2. For all $k\in\{1,\ldots,K\}$, find the maximum in $\mu_k$ of $Q(\,\cdot\,\vert\, \theta^{(t)})$.

_Recall that for any symmetric matrix $A\in\mathcal{S}_d^+$ and any vector $u\in\mathbb{R}^d$, the gradient of ${u^\top}Au$ with respect to $u$ is $2Au$._

3. For all $k\in\{1,\ldots,K\}$, find the maximum in $\Sigma_k$ of $Q(\,\cdot\,\vert\, \theta^{(t)})$. 

_Recall that for any positive symmetric matrix $A\in\mathcal{S}_d^+$, the gradient of $\log\vert A\vert$ with respect to $A$ is $A^{-1}$. Moreover, for any vectors $u,v\in\mathbb{R}^d$ the gradient of ${u^\top}Av$ with respect to $A$ is the matrix $u{v^\top}$._
		
4. Interpret those quantities.

> **Exercise**

##### <span style="color:purple">**Todo:** Write a 'm_step' function to perform the M-step.</span>

In [8]:
## TO BE COMPLETED ##

def m_step(X, T):
    """
        M step of the EM algorithm, caculates the MLE of taus, mus and sigmas
        :param X: Nxn matrix, the dataset, N number of n-dim data points
        :param T: KxN matrix, the T matrix is the posterior matrix where the i, j th component is the T_{k, i}
        :return: a 3-tuple:
            - taus: K-dim array, the estimated prior probability for each hidden variable z
            - mus: Kxn matrix, the estimated mean of the n-dim gaussian component, for each of the k component
            - sigmas: Kxnxn matrix, the covariance matrix of the n-dim gaussian component, for each of the k component
    """

    [...]

In [9]:
# %load solutions/m_step.py

Given you codes, an EM-training loop writes

In [10]:
%%script echo skipping

for i in range(100):
    T = e_step(X, taus, mus, sigmas)
    sigmas_prev = sigmas
    taus, mus, sigmas = m_step(X, T)
    if np.min(abs(sigmas - sigmas_prev) < 0.1):
        print(f"break after {i}th iteration")
        break

skipping


## Training

We put all this function in a "big" class for a more convenient use.

In [13]:
class GaussianMixture():
    def __init__(self, n_hidden=2, max_iter=100, seed=None):
        """
            :param n_hidden: number of hidden variables (or the number of components)
            :param max_iter: maximum EM iteration allowed, default to 100
            :param seed: the random seed for initialisation
        """
        self.n_hidden = n_hidden
        self.max_iter = max_iter
        self.seed = seed
        self.taus, self.mus, self.sigmas = None, None, None

    def fit(self, X):
        """
            :param X: Nxn matrix (N be the num of data points and n be the dimension of each data point)
        """
        n_hid = self.n_hidden
        n_var = X.shape[1]

        np.random.seed(self.seed) # setup seed, if None means no seed
        mus = np.random.randn(n_hid, n_var)*10 # initialise means of k components
        np.random.seed(self.seed)
        # initialise  sigmas of k components with identity
        sigmas = np.array([np.eye(n_var) for _ in range(n_hid)])
        taus = np.ones(n_hid)/n_hid # assume uninformative prior

        for i in range(self.max_iter):
            T = e_step(X, taus, mus, sigmas)
            sigmas_prev = sigmas
            taus, mus, sigmas = m_step(X, T)
            if np.min(abs(sigmas - sigmas_prev) < 0.1):
                print(f"break after iteration {i+1}")
                break
        self.taus, self.mus, self.sigmas = taus, mus, sigmas
        return self

    def predict_proba(self, X):
        T = e_step(X, self.taus, self.mus, self.sigmas)
        return T.T # transpose, so it's 1st dimension matches the X's dimension N.

    def predict(self, X):
        T = e_step(X, self.taus, self.mus, self.sigmas)
        return np.argmax(T.T, axis=1)

##### <span style="color:purple">**Todo:** Comment on the `predict_proba` and `predict` functions.</span>

What can they achieve?

> Answer

##### <span style="color:purple">**Todo:** Test your algorithm</span>

1. Generate a dataset
2. Fit a Gaussian Mixture to your data
3. Discuss

In [15]:
## TO BE COMPLETED ##
# > Test the model <


## SAEM Algorithm

##### <span style="color:purple">✍️ **Todo:** Justify that the Gaussian mixture model belongs to the exponential family</span>

1. Justify that the Gaussian mixture model belongs to the exponential family
2. Provide sufficient statistics
3. What happens to the M-step in this case?